In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
# Import necessary modules from the transformers library for tokenizing text and loading a pre-trained model
import torch                  # Import PyTorch for tensor operations and model inference
import requests               # Import requests to fetch webpage data
from bs4 import BeautifulSoup # Import BeautifulSoup for parsing HTML content
import re                     # Import regex module for pattern matching in HTML content
import pandas as pd           # Import pandas for data manipulation and creating DataFrame
import numpy as np            # Import numpy to convert list into array format for DataFrame

tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
# Load the pre-trained tokenizer for sentiment analysis
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
# Load the pre-trained sentiment analysis model

r = requests.get('https://www.flipkart.com/marq-flipkart-80cm-32-inch-hd-ready-led-tv/product-reviews/itmewmcbpqhht4df?pid=TVSEWMCBXTP4PJ3J')              # Send a GET request to Flipkart product review page and store the response
soup = BeautifulSoup(r.text, 'html.parser')  # Parse the HTML content of the page using BeautifulSoup
regex = re.compile('.*ZmyHeo.*')  # Compile a regex pattern to match specific class names in review divs
results = soup.findAll('div', {'class':regex})  # Find all div elements matching the review class using the regex
reviews = [result.text for result in results]  # Extract and store text content of each review div into a list
df = pd.DataFrame(np.array(reviews), columns=['review'])  # Create a pandas DataFrame from the list of reviews

# Define a function to analyze sentiment of a given review text
def sentiment_analyser(review):
    tokens = tokenizer.encode(review, return_tensors="pt") # Tokenize the input review text and return PyTorch tensors
    result = model(tokens) # Pass the tokens through the model to get the sentiment prediction logits
    return int(torch.argmax(result.logits))+1 # Return the predicted sentiment score (1 to 5) based on highest logit

df['sentiment'] = df['review'].apply(lambda x: sentiment_analyser(x[:512]))
# Apply the sentiment_analyser function to each review (limited to first 512 characters) and store results in a new 'sentiment' column
df
# Display the DataFrame with reviews and their corresponding sentiment scores

,review,sentiment
0,Its a very nice TV good sould and TV panel is ...,3
1,First thing first congratulations flipcart for...,4
2,nice product value for moneyREAD MORE,4
3,Great quality delivered..READ MORE,5
4,very good product...same day installation.all ...,5
5,"Good product, I am using as monitor for my PC,...",4
6,Very nice looking.Sound quality is very good.P...,5
7,Overall Tv is Good.Picture Quality is better a...,4
8,24 inch at this price was great at time of pur...,5
9,Very good product at this price and the servic...,5
